In [ ]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console

In [ ]:
model = OpenAIChatCompletionClient(model="gpt-5-nano")
clarity_agent = AssistantAgent(
    "ClarityAgent",
    model_client=model,
    system_message="""You are an expert editor focused on clarity and simplicity.
    Your job is to eliminate ambiguity, redundancy, and make every sentence crisp and clear.
    Don't worry about persuasion or tone -- just make the message easy to read and understand."""
)
tone_agent=AssistantAgent(
    "ToneAgent",
    model_client=model,
    system_message="""You are a communication coach focused on emotional tone and professionalism.
    Your job is to make the email sound warm, confident, and human -- while staying professional and appropriate for the audience.
    Improve the emotional resonance, polish the phrasing, and adjust any words that may come off as stiff, cold, or overly casual."""
)
persuasion_agent=AssistantAgent(
    "PersuasionAgent",
    model_client=model,
    system_message="""You are a persuasion expert trained in marketing, behavioral psychology, and copywriting.
    Your job is to enhance the email's persuasive power: improve call to action, structure arguments, emphasize benefits.
    Remove weak or passive language."""
)
synthesizer_agent=AssistantAgent(
    "SynthesizerAgent",
    model_client=model,
    system_message="""
    You are an advanced email-writing specialist.
    Your role is to read all prior agent responses and revisions, and then **synthesize the best ideas** into a unified, polished draft of the email.
    Focus on: Integrating clarity, tone, and persuasion improvements.
    Ensuring coherence, fluency, and a natural voice; Creating a version that feels professional, effective, and readable.
    """
)
critic_agent=AssistantAgent(
    "CriticAgent",
    model_client=model,
    system_message="""You are an email quality evaluator. 
    Your job is to perform a final review of the synthesized email and determine if it meets professional standards.
    Review the email for:
    -Clarity and flow, appropriate professional tone, effective call-to-action, and overall coherence.
    -Be constructive but decisive. If the email has major flaws (unclear message, unprofessional tone, or missing key elements), provide ONE specific improvement suggestion.
    If the email meets professional standards and communicate effectively, respond with 'The email meets professional standards.' followed by 'TERMINATE' on a new line.
    You should approve emails that are good enough for professional use, even if not perfect.

    """
)

In [6]:
text_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=20)

termination_condition = text_termination | max_messages_termination

In [7]:
team = RoundRobinGroupChat(
    participants=[clarity_agent, tone_agent, persuasion_agent, synthesizer_agent, critic_agent],
    termination_condition=termination_condition)

await Console( #asynchronous Python을 쓰기 때문
    team.run_stream(task="Hi! I'm hungry, buy me lunch and invest in my company. Thanks.")
)

---------- TextMessage (user) ----------
Hi! I'm hungry, buy me lunch and invest in my company. Thanks.
---------- TextMessage (ClarityAgent) ----------
Please buy me lunch and invest in my company.
---------- TextMessage (ToneAgent) ----------
Subject: Lunch to discuss an investment opportunity

Hi [Name],

I hope you’re doing well. I’m reaching out to see if you’d be open to meeting for lunch to discuss a potential investment in my company, [Company Name]. We’re building [one-line description of product/service and traction—e.g., “a marketplace that connects X with Y and has grown to Z users in 12 months”], and I’d love to share our progress and explore whether this might be a fit for you.

Are you available for lunch any time next week? I’m free on [day/time option 1] or [day/time option 2], but happy to work around your schedule and meet wherever is most convenient. If it helps, I can send a short deck in advance.

Thanks for considering—I appreciate your time and look forward to h

TaskResult(messages=[TextMessage(id='12bb0b3c-e6e7-40f7-b3e0-8a943d411dd8', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 1, 9, 22, 37, 40, 35395, tzinfo=datetime.timezone.utc), content="Hi! I'm hungry, buy me lunch and invest in my company. Thanks.", type='TextMessage'), TextMessage(id='722b8e33-83be-4f38-ad33-f8ee24876b55', source='ClarityAgent', models_usage=RequestUsage(prompt_tokens=74, completion_tokens=275), metadata={}, created_at=datetime.datetime(2026, 1, 9, 22, 37, 52, 396288, tzinfo=datetime.timezone.utc), content='Please buy me lunch and invest in my company.', type='TextMessage'), TextMessage(id='f6dba53e-0e55-4018-b8cc-af973404285f', source='ToneAgent', models_usage=RequestUsage(prompt_tokens=110, completion_tokens=908), metadata={}, created_at=datetime.datetime(2026, 1, 9, 22, 38, 7, 313177, tzinfo=datetime.timezone.utc), content='Subject: Lunch to discuss an investment opportunity\n\nHi [Name],\n\nI hope you’re doing well. I’m reachi